# Install necessary libraries


In [27]:
!pip install google-generativeai
!pip install python-dotenv
!pip install langchain
!pip install PyPDF2
!pip install faiss-cpu
!pip install langchain_google_genai

### Get Google API keys from : https://aistudio.google.com/app/apikey

# Import Required Libraries


In [29]:
from google.colab import files
import os
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

# set API key as environment variable

In [30]:
GOOGLE_API_KEY = "YOUR_GOOGLE_API_KEY"
load_dotenv()
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Run the Model

In [33]:
def get_pdf_text(pdf_docs):
    text=""
    for pdf in pdf_docs:
        pdf_reader= PdfReader(pdf)
        for page in pdf_reader.pages:
            text+= page.extract_text()
    return  text

def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks

def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")

def get_conversational_chain():
    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

    model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3)

    prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain

def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

    new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    docs = new_db.similarity_search(user_question)

    chain = get_conversational_chain()

    response = chain.invoke(
        {"input_documents":docs, "question": user_question}
        , return_only_outputs=True)

    print(response)
    print("Reply: ", response["output_text"])


def main():
    print("Chat with PDF using Gemini💁")

    print("Menu:")
    uploaded = files.upload()
    pdf_docs = list(uploaded.keys())
    if pdf_docs:
        print("Processing...")
        raw_text = get_pdf_text(pdf_docs)
        text_chunks = get_text_chunks(raw_text)
        get_vector_store(text_chunks)
        print("Done")

    while True:
        user_question = input("Ask a Question from the PDF Files (or type 'exit' to quit): ")

        if user_question.lower() == 'exit':
            break

        if user_question:
            user_input(user_question)

if __name__ == "__main__":
    main()

Chat with PDF using Gemini💁
Menu:


Saving Oil and gas production handbook.pdf to Oil and gas production handbook.pdf
Processing...
Done
Ask a Question from the PDF Files (or type 'exit' to quit): what id Gas processing?
{'output_text': 'Raw natural gas must be processed to meet the trading specifications of pipeline and gas distribution companies.  As part of the purification other components such as NGL is produced, and pollutants extracted.'}
Reply:  Raw natural gas must be processed to meet the trading specifications of pipeline and gas distribution companies.  As part of the purification other components such as NGL is produced, and pollutants extracted.
Ask a Question from the PDF Files (or type 'exit' to quit): exit
